In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1131683,2021-03-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1131684,2021-03-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1131685,2021-03-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1131686,2021-04-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30435,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30437,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30439,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30441,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30443,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
698433,2021-03-29,Arkansas,Arkansas,5001,2072,36.00,5000,Arkansas,AR,Arkansas,State,3017804
698435,2021-03-30,Arkansas,Arkansas,5001,2076,36.00,5000,Arkansas,AR,Arkansas,State,3017804
698437,2021-03-31,Arkansas,Arkansas,5001,2076,36.00,5000,Arkansas,AR,Arkansas,State,3017804
698439,2021-04-01,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1131683,2021-03-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1131684,2021-03-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1131685,2021-03-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1131686,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-02') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
437,2021-04-02,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88
872,2021-04-02,Cook,Illinois,500914,10242.00,17031,IL,County,5150233,9726.05,198.86
1306,2021-04-02,Orange,California,266464,4761.00,6059,CA,County,3175692,8390.74,149.92
1739,2021-04-02,Maricopa,Arizona,525002,9670.00,4013,AZ,County,4485414,11704.65,215.59
2172,2021-04-02,Los Angeles,California,1221506,23235.00,6037,CA,County,10039107,12167.48,231.44
...,...,...,...,...,...,...,...,...,...,...,...
1131123,2021-04-02,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1131295,2021-04-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1131436,2021-04-02,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1131573,2021-04-02,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
437,2021-04-02,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88,67.88,3965.54
872,2021-04-02,Cook,Illinois,500914,10242.00,17031,IL,County,5150233,9726.05,198.86,198.86,9726.05
1306,2021-04-02,Orange,California,266464,4761.00,6059,CA,County,3175692,8390.74,149.92,149.92,8390.74
1739,2021-04-02,Maricopa,Arizona,525002,9670.00,4013,AZ,County,4485414,11704.65,215.59,215.59,11704.65
2172,2021-04-02,Los Angeles,California,1221506,23235.00,6037,CA,County,10039107,12167.48,231.44,231.44,12167.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131123,2021-04-02,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1131295,2021-04-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1131436,2021-04-02,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1131573,2021-04-02,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
730258,2021-04-02,Hale,Alabama,2167,73.00,1065,AL,County,14651,14790.80,498.26,498.26,14790.80,1
794753,2021-04-02,Clarke,Alabama,3453,60.00,1025,AL,County,23622,14617.73,254.00,254.00,14617.73,2
571581,2021-04-02,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3
425436,2021-04-02,Franklin,Alabama,4187,82.00,1059,AL,County,31362,13350.55,261.46,261.46,13350.55,4
506885,2021-04-02,Etowah,Alabama,13650,344.00,1055,AL,County,102268,13347.28,336.37,336.37,13347.28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697331,2021-04-02,Sublette,Wyoming,710,7.00,56035,WY,County,9831,7222.05,71.20,71.20,7222.05,19
696960,2021-04-02,Converse,Wyoming,983,17.00,56009,WY,County,13822,7111.85,122.99,122.99,7111.85,20
866125,2021-04-02,Lincoln,Wyoming,1357,12.00,56023,WY,County,19830,6843.17,60.51,60.51,6843.17,21
963681,2021-04-02,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
571581,2021-04-02,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3,1
730258,2021-04-02,Hale,Alabama,2167,73.00,1065,AL,County,14651,14790.80,498.26,498.26,14790.80,1,2
243338,2021-04-02,Walker,Alabama,7066,274.00,1127,AL,County,63521,11123.88,431.35,431.35,11123.88,28,3
616338,2021-04-02,Greene,Alabama,904,34.00,1063,AL,County,8111,11145.36,419.18,419.18,11145.36,27,4
570835,2021-04-02,Crenshaw,Alabama,1502,57.00,1041,AL,County,13772,10906.19,413.88,413.88,10906.19,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614849,2021-04-02,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
866125,2021-04-02,Lincoln,Wyoming,1357,12.00,56023,WY,County,19830,6843.17,60.51,60.51,6843.17,21,20
844590,2021-04-02,Uinta,Wyoming,2139,12.00,56041,WY,County,20226,10575.50,59.33,59.33,10575.50,4,21
242198,2021-04-02,Teton,Wyoming,3624,9.00,56039,WY,County,23464,15444.94,38.36,38.36,15444.94,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129023,2021-03-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1129024,2021-03-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1129025,2021-03-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1129026,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
727230,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14790.80,2,1,1.00
727231,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14790.80,2,1,0.00
727232,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14790.80,2,1,0.00
727233,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14790.80,2,1,0.00
727234,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14790.80,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880569,2021-03-29,Crook,Wyoming,424,11.00,56011,WY,County,7584,5590.72,145.04,158.23,5590.72,6,23,2.00
880570,2021-03-30,Crook,Wyoming,424,11.00,56011,WY,County,7584,5590.72,145.04,158.23,5590.72,6,23,0.00
880571,2021-03-31,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
880572,2021-04-01,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
569291,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,1.00,0.00
569292,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
569293,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
569294,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
569295,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567421,2021-03-29,Albany,Wyoming,3968,11.00,56001,WY,County,38880,10205.76,28.29,28.29,10246.91,23,6,7.00,0.00
567422,2021-03-30,Albany,Wyoming,3969,11.00,56001,WY,County,38880,10208.33,28.29,28.29,10246.91,23,6,1.00,0.00
567423,2021-03-31,Albany,Wyoming,3970,11.00,56001,WY,County,38880,10210.91,28.29,28.29,10246.91,23,6,1.00,0.00
567424,2021-04-01,Albany,Wyoming,3979,11.00,56001,WY,County,38880,10234.05,28.29,28.29,10246.91,23,6,9.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-02') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
290426,2021-04-02,Imperial,California,27520,711.00,6025,CA,County,181215,15186.38,392.35,392.35,15186.38,1,2,14.00,0.00
2172,2021-04-02,Los Angeles,California,1221506,23235.00,6037,CA,County,10039107,12167.48,231.44,231.44,12167.48,2,5,606.00,46.00
144724,2021-04-02,San Bernardino,California,291199,4404.00,6071,CA,County,2180085,13357.23,202.01,202.01,13357.23,3,4,177.00,48.00
508333,2021-04-02,Inyo,California,1374,36.00,6027,CA,County,18039,7616.83,199.57,199.57,7616.83,4,25,1.00,1.00
64769,2021-04-02,Stanislaus,California,59271,1003.00,6099,CA,County,550660,10763.63,182.15,182.15,10763.63,5,9,258.00,2.00
30042,2021-04-02,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,119.00,23.00
80286,2021-04-02,Tulare,California,49100,817.00,6107,CA,County,466195,10532.07,175.25,175.25,10532.07,7,10,29.00,1.00
51545,2021-04-02,San Joaquin,California,70117,1310.00,6077,CA,County,762148,9199.92,171.88,171.88,9199.92,8,19,139.00,10.00
29258,2021-04-02,Fresno,California,99317,1616.00,6019,CA,County,999101,9940.64,161.75,161.75,9940.64,9,14,77.00,0.00
460725,2021-04-02,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,38.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1073743,2021-04-02,Lassen,California,5652,1.00,6035,CA,County,30573,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00
290426,2021-04-02,Imperial,California,27520,711.00,6025,CA,County,181215,15186.38,392.35,392.35,15186.38,1,2,14.00,0.00
616280,2021-04-02,Kings,California,22659,241.00,6031,CA,County,152940,14815.61,157.58,157.58,14815.61,11,3,10.00,0.00
144724,2021-04-02,San Bernardino,California,291199,4404.00,6071,CA,County,2180085,13357.23,202.01,202.01,13357.23,3,4,177.00,48.00
2172,2021-04-02,Los Angeles,California,1221506,23235.00,6037,CA,County,10039107,12167.48,231.44,231.44,12167.48,2,5,606.00,46.00
30042,2021-04-02,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,119.00,23.00
181917,2021-04-02,Kern,California,106329,1277.00,6029,CA,County,900202,11811.68,141.86,141.86,11811.68,14,7,41.00,13.00
460725,2021-04-02,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,38.00,1.00
64769,2021-04-02,Stanislaus,California,59271,1003.00,6099,CA,County,550660,10763.63,182.15,182.15,10763.63,5,9,258.00,2.00
80286,2021-04-02,Tulare,California,49100,817.00,6107,CA,County,466195,10532.07,175.25,175.25,10532.07,7,10,29.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
290426,2021-04-02,Imperial,California,27520,711.00,6025,CA,County,181215,15186.38,392.35,392.35,15186.38,1,2,14.00,0.00
2172,2021-04-02,Los Angeles,California,1221506,23235.00,6037,CA,County,10039107,12167.48,231.44,231.44,12167.48,2,5,606.00,46.00
144724,2021-04-02,San Bernardino,California,291199,4404.00,6071,CA,County,2180085,13357.23,202.01,202.01,13357.23,3,4,177.00,48.00
508333,2021-04-02,Inyo,California,1374,36.00,6027,CA,County,18039,7616.83,199.57,199.57,7616.83,4,25,1.00,1.00
64769,2021-04-02,Stanislaus,California,59271,1003.00,6099,CA,County,550660,10763.63,182.15,182.15,10763.63,5,9,258.00,2.00
30042,2021-04-02,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,119.00,23.00
80286,2021-04-02,Tulare,California,49100,817.00,6107,CA,County,466195,10532.07,175.25,175.25,10532.07,7,10,29.00,1.00
51545,2021-04-02,San Joaquin,California,70117,1310.00,6077,CA,County,762148,9199.92,171.88,171.88,9199.92,8,19,139.00,10.00
29258,2021-04-02,Fresno,California,99317,1616.00,6019,CA,County,999101,9940.64,161.75,161.75,9940.64,9,14,77.00,0.00
460725,2021-04-02,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,38.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15186.38,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3802,03/29/21,Lassen,5653,1.00,18490.17,3.27,3.27,18486.90,56,1,4.00,0.00
3803,03/30/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,-1.00,0.00
3804,03/31/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00
3805,04/01/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15186.38,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15186.38,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3888,03/29/21,Merced,30521,435.00,10991.43,156.66,158.10,11051.21,10,8,102.00,1.00
3889,03/30/21,Merced,30558,435.00,11004.75,156.66,158.10,11051.21,10,8,37.00,0.00
3890,03/31/21,Merced,30593,436.00,11017.36,157.02,158.10,11051.21,10,8,35.00,1.00
3891,04/01/21,Merced,30649,438.00,11037.53,157.74,158.10,11051.21,10,8,56.00,2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)